## 라이브러리 선언 및 데이터 불러오기

In [ ]:
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 머신러닝 라이브러리
from sklearn import tree
from sklearn import ensemble

In [ ]:
featuresData = pd.read_csv("../dataset/feature_regression_example.csv")

# A. 데이터 전처리

### A-1. 타입통합 / 특성 숫자컬럼 추가

In [ ]:
#데이터 타입 통합
featuresData["REGIONID"] = featuresData["REGIONID"].astype(str)
featuresData["PRODUCTGROUP"] = featuresData["PRODUCTGROUP"].astype(str)

In [ ]:
# 특성값 숫자컬럼 변경

In [ ]:
ynMap["N"]

In [ ]:
featuresData["HOLIDAY_YNMAP"] = featuresData.HOLIDAY.map(ynMap)  ##NP.WHERE를 사용하는 대신 MAP을 사용한다.

In [ ]:
#변환대상이 100개 된다?(labelencode)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
ynLabelEn = LabelEncoder()

In [ ]:
ynLabelEn.fit_transform(featuresData["HOLIDAY"])

In [ ]:
featuresData.drop_duplicates()

In [ ]:
csData = pd.read_csv("../dataset/customerdata.csv")
csData

In [ ]:
csData["CUSTID"] = ynLabelEn.fit_transform(csData["CUSTID"])  ## 1. custtype의 갯수를 확인하고 2.오름차순으로 정렬한 후에 3.숫자로 변경해줌.

In [ ]:
csData

In [ ]:
ynLabelEn.inverse_transform(csData["CUSTID"])

In [ ]:
featuresData.head(1)

In [ ]:
# 숫자가 아니라 문자를 머신러닝에 던지면 에러가 발생함.(변경 Y=1 , N=0)
featuresData["PROMOTION_LB"] = np.where(featuresData.PROMOTION =="Y",1,0)
featuresData["HOLIDAY_LB"] = np.where(featuresData.HOLIDAY =="Y",1,0)

In [ ]:
# 검증
featuresData.loc[(featuresData.PROMOTION_LB == 1) &
                 (featuresData.PROMOTION == "Y") ] 

## A-2 특성 선정 / 데이터 분리

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns #만약 오류가 안되면 pip uninstall scipy 하고 pip install scipy 하면 됌(scipy는 통계를 담당하는 라이브러리임)
%matplotlib inline  ## 

In [ ]:
sns.distplot(featuresData["QTY"]) #distplot = 변수하나에 대한 분포도

In [ ]:
sns.lmplot(data = featuresData, x = "YEAR", y = "QTY") # 상관계수를 나타낸다.  HCLUS : 블랙프라이데이와 같은 큰 휴일은 0으로 지정해놨음.
plt.show()

In [ ]:
stdCorrRepeat = np.array( list( range(3,8,1) ) ) / 10  ##0.3~0.8의 상관계수가 0.1씩 증가한다.

In [ ]:
stdCorrRepeat

array([0.3, 0.4, 0.5, 0.6, 0.7])

# 머신러닝과 딥러닝의 차이
# 딥러닝은 재료를 모두 쓰고 너가 알아서 필요한 자료 선택해, 머신러닝은 상관계수를 도출하고 feature을 설정

In [ ]:
# 특성 선정
corrDf = featuresData.corr() #판매량에 영향을 주는 요소들이 어떤거인지 알 수 있음. #보통 0.3~0.7 은 상관관계가 있고, 0.7이상은 뚜렷한 상관관계를 띄고있다.
stdCorr = 0.3

In [ ]:
corrDf

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT
YEARWEEK,1.000000,0.956598,0.275593,0.112267,-0.071586,0.404889
YEAR,0.956598,1.000000,-0.016493,0.028931,0.028593,0.321193
WEEK,0.275593,-0.016493,1.000000,0.289766,-0.339943,0.329705
QTY,0.112267,0.028931,0.289766,1.000000,-0.537230,0.700195
HCLUS,-0.071586,0.028593,-0.339943,-0.537230,1.000000,-0.545619
PRO_PERCENT,0.404889,0.321193,0.329705,0.700195,-0.545619,1.000000


In [ ]:
qtyCorr = corrDf.loc[["QTY"] ]
qtyCorr

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT
QTY,0.112267,0.028931,0.289766,1.0,-0.53723,0.700195


In [ ]:
qtyCorr = corrDf.loc[:,["QTY"] ].sort_values(by=["QTY"],ascending=False) #내림차순 false 오름차순 true
qtyCorr

,QTY
QTY,1.000000
PRO_PERCENT,0.700195
WEEK,0.289766
YEARWEEK,0.112267
YEAR,0.028931
HCLUS,-0.537230


In [ ]:
sns.heatmap( qtyCorr, annot = True )
#각각의 지표에 대해서 직접 숫자를 나타내줌. sort를 해주면 기존의 입력되어있는 순서로 보여주는데 숫자가 큰수로 정렬되어 더 명확하게 볼수 있음.

NameError: name 'sns' is not defined

In [ ]:
# stepwise
# 점진 선택법...

In [ ]:
features = list( corrDf.loc[ (abs(corrDf.QTY) >= stdCorr) & (corrDf.QTY != 1) ].index ) ##나자신인 qty는 답안지 이기 때문에 제외시켜줘야한다.인덱스는 숫자 문자 둘다 가능함.

In [ ]:
features

['HCLUS', 'PRO_PERCENT']

In [ ]:
label = ["QTY"]

In [ ]:
# 데이터 분리(7:3)

In [ ]:
stdRatio = 0.7

In [ ]:
featuresData.shape

(105, 12)

In [ ]:
# 원래는 시간의 순으로 데이터를 7대3으로 나누나 시간이 없는 데이터도 있을수 있기 때문에 무작위(랜덤)로 뽑아서 7대3 데이터를 나눈다.

### 1.랜덤 데이터분리(training data / test data)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
traininDataFeatures,\
testDataFeatures,\
trainingDataLabel,\
testDataLabel = train_test_split(featuresData.loc[:,features],
                                 featuresData.loc[:,label], 
                                 test_size=0.2, 
                                 random_state = 10) 
#계속돌릴때마다 값이 계속 달라진다 그래서 한번 추출한 데이터를 픽스하고 싶으면 random_state=10을 적어준다.
traininDataFeatures,\
testDataFeatures,\
trainingDataLabel,\
testDataLabel 

(     HCLUS  PRO_PERCENT
 96       0     0.421888
 68       4     0.209442
 38       4     0.000000
 52       1     0.237768
 74       1     0.280258
 ..     ...          ...
 28       1     0.280258
 64       4     0.209442
 15       4     0.000000
 100      0     0.421888
 9        4     0.000000
 
 [84 rows x 2 columns],
     HCLUS  PRO_PERCENT
 85      4     0.000000
 35      1     0.237768
 48      0     0.357511
 47      0     0.357511
 82      4     0.280258
 42      4     0.209442
 58      4     0.237768
 80      1     0.308584
 3       2     0.208155
 41      2     0.209442
 2       4     0.208155
 26      1     0.280258
 43      4     0.000000
 14      4     0.000000
 60      4     0.237768
 45      0     0.357511
 19      1     0.209442
 1       4     0.209442
 97      0     0.421888
 79      1     0.308584
 71      4     0.000000,
       QTY
 96   1615
 68    588
 38    980
 52   1750
 74   2282
 ..    ...
 28   2671
 64   1707
 15    375
 100  2412
 9     350
 
 [84 rows x

In [ ]:
print(trData.shape)
print(tdData.shape)
print(c.shape)
print(d.shape)

NameError: name 'trData' is not defined

## 2.시간순 데이터분리(training data / test data)

In [ ]:
sortKey = ["YEARWEEK"]

In [ ]:
sortedData =featuresData.sort_values(sortKey).reset_index(drop=True)
sortedData.YEARWEEK.describe()  # 예를 들어 2015년 데이터를 가지고 트레이닝데이터를 만들어보고 2016년데이터를 가지고 테스트해보자!

count       105.000000
mean     201576.276190
std          52.254278
min      201501.000000
25%      201527.000000
50%      201553.000000
75%      201626.000000
max      201652.000000
Name: YEARWEEK, dtype: float64

In [ ]:
stdYearweek = 2016

In [ ]:
trainingDataFeatures = featuresData.loc[featuresData.YEAR < stdYearweek, features] #문제지
trainingDataLabel = featuresData.loc[featuresData.YEAR < stdYearweek, label] #정답
testDataFeatures = featuresData.loc[featuresData.YEAR > stdYearweek, features] #문제지
testDataLabel = featuresData.loc[featuresData.YEAR > stdYearweek, label] #정답

In [ ]:
stdIndex = int(featuresData.shape[0] * stdRatio)
stdIndex

73

In [ ]:
trainingDataFeatures = featuresData.loc[0:stdIndex, features] #문제지
trainingDataLabel = featuresData.loc[0:stdIndex, label] #정답
testDataFeatures = featuresData.loc[stdIndex+1 :, features] #문제지
testDataLabel = featuresData.loc[stdIndex+1 :, label] #정답

In [ ]:
print ( trainingDataFeatures.shape )
print ( trainingDataLabel.shape )
print ( testDataFeatures.shape )
print ( testDataLabel.shape )

(74, 2)
(74, 1)
(31, 2)
(31, 1)


## B 모델적용

### B-3. 모델적용

In [ ]:
from sklearn import tree
from sklearn.tree import plot_tree

In [ ]:
# 알고리즘을 고정시킴으로써 예측값을 재현 가능하게 해줌(random_state=10)
dtModel = tree.DecisionTreeRegressor(random_state=10)

In [ ]:
dtModel.fit(X=trainingDataFeatures,
            y=trainingDataLabel) ## 대문자 X는 문제지, 소문자 y는 정답지)

NameError: name 'trainingDataFeatures' is not defined

In [ ]:
plt.figure(figsize=(13,8))
plot_tree(dtModel)

## C. 예측/검증

### c-1. 예측

In [ ]:
dtModel

DecisionTreeRegressor(random_state=10)

In [ ]:
predictValue = dtModel.predict( X = testDataFeatures ) ##테스트데이터(X)의 문제지를 대입하였을때 정답지(predictValue)를 추출한다.

In [ ]:
len(predictValue) ##qty(31개)가 정답지로 추출된다.

31

In [ ]:
import pandas as pd

In [ ]:
ModelDf = pd.read_excel("../dataset/모델.xlsx")

In [ ]:
ModelDf

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,모델,ITEM,연주차정보,정답지,예측값
1,A,A정석카드,202310,100,50
2,A,A정석카드,202311,100,50
3,A,A정석카드,202312,100,50
4,A,A정석카드,202313,100,50
5,A,A정석카드,202314,100,50
6,B,A정석카드,202310,100,100
7,B,A정석카드,202311,100,100
8,B,A정석카드,202312,100,100
9,B,A정석카드,202313,100,100


# 미래는 어떻게 예측할 것이냐?

In [ ]:
features

['HCLUS', 'PRO_PERCENT', 'PROMOTION_LB', 'HOLIDAY_LB']

In [ ]:
inputHclus = 1 # 대휴일1 소휴일 :4
inputProPercent = 0.5 # 프로모션 비율
inputPromotionLb = 1 # 프로모션 적용 Y
inputHlb = 1 # 홀리데이 적용 Y

In [ ]:
testData = pd.DataFrame( [ [inputHclus, inputProPercent, inputPromotionLb, inputHlb] ] )

In [ ]:
dtModel.predict(testData)

C:\Users\SMART27\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([2193.8])

In [ ]:
import pandas as pd
import pickle
data450 = pd.read_csv("../dataset/kopo_450data.csv")

data450.shape

### 피클 파일 저장하기 (바이너리) ###
with open("dt0522.pickle","wb") as fw:
    pickle.dump(dtModel,fw)